In [8]:
import os
os.sys.path

# Installing libraries
!pip3 install opencv-python
!pip3 install numpy
!pip3 install pandas
!pip3 install panel

# Importing
import numpy as np
import pandas as pd
from matplotlib.figure import Figure
import panel as pn
from panel.interact import interact
import param
import matplotlib.pyplot as plt
import hvplot.pandas
pn.extension()


# Reading Excel files and creation of different Dfs
df = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='WorkstageRecordItem')
df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
df2 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobOrder')

df = df.drop(columns =["Id","JobOrderId","Status","Reference","TechnicianSignature_StampNo","ApproverRole","ReleaseToSignatorySignature_StampNo","TechnicianSignature_SignedOn","CompletedByUserId","CompletedByFullname","ApprovedByUserId","ApprovedByFullname","ApprovedOn","ReleaseToSignatorySignature_DigitalCertificateId","ReleaseToSignatorySignature_Signature","ReleaseToSignatorySignature_SignedByUserId","ReleaseToSignatorySignature_SignedOn","TechnicianSignature_DigitalCertificateId","TechnicianSignature_Signature","TechnicianSignature_SignedByUserId"], axis = 1)
df1 = df1.drop(["MWC"], axis = 1)
df2 = df2.drop(["MWC","ReviewerSignature_Signature","ComplianceDeclarationRecord_ReleaseToSignatorySignature_Signature","SignWorkscopeAndCRRBy","WorkScopeId","SignWorkscopeAndCRRBy","TechnicianSignature_StampNo","TechnicianSignature_SignedOn","TechnicianSignature_SignedByUserId","TechnicianSignature_Signature","TechnicianSignature_DigitalCertificateId","ReviewerSignature_StampNo","ReviewerSignature_SignedOn","ReviewerSignature_SignedByUserId","ReviewerSignature_DigitalCertificateId","ComplianceDeclarationRecord_TechnicianSignature_StampNo","ComplianceDeclarationRecord_ReleaseToSignatorySignature_StampNo","ComplianceDeclarationRecord_TechnicianSignature_SignedOn","ComplianceDeclarationRecord_TechnicianSignature_SignedByUserId","ComplianceDeclarationRecord_TechnicianSignature_Signature","ComplianceDeclarationRecord_TechnicianSignature_DigitalCertificateId","ComplianceDeclarationRecord_ReleaseToSignatorySignature_SignedOn","ComplianceDeclarationRecord_ReleaseToSignatorySignature_SignedByUserId","ComplianceDeclarationRecord_ReleaseToSignatorySignature_DigitalCertificateId","RowVersion","ComplianceDeclarationRecord_ReviewStatus","ComplianceDeclarationRecord_LastDeclaredOn","RejectReason","ComplianceDeclarationRecord_LastDeclaredByUserId","ComplianceDeclarationRecord_LastDeclaredByFullName","ComplianceDeclarationRecord_LastApprovedOn","ComplianceDeclarationRecord_LastApprovedByUserId","ComplianceDeclarationRecord_LastApprovedByFullName","ComplianceDeclarationRecord_IsDeclared","ReviewedBy","NSN","SignWorkscopeAndCRRSignature_DigitalCertificateId","TechnicianSignature_StampNo","MPGType","SignWorkscopeAndCRRSignature_Signature","SignWorkscopeAndCRRSignature_SignedByUserId","SignWorkscopeAndCRRSignature_SignedOn","SignWorkscopeAndCRRSignature_StampNo"], axis = 1)

# Seperation of date and time
df['Dates'] = pd.to_datetime(df['CompletedOn'], format = '%Y%m%d').dt.date
df['Time'] = pd.to_datetime(df['CompletedOn'], format = '%H%m%s').dt.time

# Sorting Values based on Job Order Number
df = df.sort_values(by=['JobOrderNo','Sequence'], ascending = True)
df1 = df1.rename(columns ={'JobOrder':'JobOrderNo'})

#Merging Excel Sheets based on Job Order Number
df3 = pd.merge(df, df1, left_on= 'JobOrderNo', right_on ='JobOrderNo')
df4 = pd.merge(df3,df2, left_on = 'JobOrderNo', right_on = 'JobOrderNo')


# Creating new dfs
df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
Cost_df = df1.drop(['JobOrder','MWC','PartDesc','Strtmlfn','Malfend','Aircraft','HrClocked'], axis = 1)
Total_Part_Number_Avail = list(Cost_df['PartNo'].unique())
Total_Work_Req_Avail = list(Cost_df['WorkReq'].unique())

# Setting up for Time
df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
Time_df = df1.drop(['JobOrder','SerialNo','MWC','PartDesc','Aircraft','ActTotCost','ActMatCost','ActLabCost','ActSubcCost','ActOthCost'], axis = 1)
New_Rows = []

Time_df['Difference'] = (Time_df['Malfend'] - Time_df['Strtmlfn']).dt.days

# Removing Start and end dates
Time_df = Time_df.drop(['Strtmlfn','Malfend'],axis = 1)


df1 = pd.read_excel(r"C:\Users\Keithon\Desktop\Capstone\Data\Test.xlsx",sheet_name='JobCost')
Cost_df = df1.drop(['JobOrder','MWC','PartDesc','Strtmlfn','Malfend','Aircraft','HrClocked'], axis = 1)
Cost_df = Cost_df.rename(columns ={'ActTotCost':'Total','ActMatCost':'Materials','ActSubcCost':'SubContract','ActLabCost':'Labour','ActOthCost':'Others'})
Cost_df = Cost_df[['PartNo','WorkReq','Labour', 'Materials', 'SubContract', 'Others', 'Total']]


# Setting up title
Title = '##Interactive Dashboard'

# Setting up panel widgets
Select_X = pn.widgets.Select(name='Part Numbers', options= Total_Part_Number_Avail)
Select_Y = pn.widgets.Select(name='Work Required', options= Total_Work_Req_Avail)
Select_A = pn.widgets.Select(name='Part Numbers', options= Total_Part_Number_Avail)
Select_B = pn.widgets.Select(name='Work Required', options= Total_Work_Req_Avail)

# Setting up plot for box
@pn.depends(Select_X,Select_Y)
def Box_Plot(Select_X,Select_Y):
    Box = Cost_df[(Cost_df['PartNo'] == Select_X) & (Cost_df['WorkReq'] == Select_Y)].hvplot.box(title = 'Part Number is:' + str(Select_X) +'\n'+'Work Performed is:'+str(Select_Y) ,value_label='Costs ($)',xlabel = 'Different Types of Cost',legend = False, width = 1100, height = 700)
    return Box


# Setting up Mahour tracking
@pn.depends(Select_A,Select_B)
def Hour_Plot(Select_A,Select_B):
    Time_Check = Time_df[(Time_df['PartNo'] == Select_A) & (Time_df['WorkReq'] == Select_B)].hvplot.box(title = 'Part Number is:' + str(Select_A) +'\n'+'Work Performed is:'+str(Select_B) ,y = 'Difference',ylabel = 'Number of Days',xlabel='Average Turn Around Days',legend = False,width = 500, height = 700)
    return Time_Check

@pn.depends(Select_A,Select_B)
def Day_Plot(Select_A,Select_B):
    ManHour = Time_df[(Time_df['PartNo'] == Select_A) & (Time_df['WorkReq'] == Select_B)].hvplot.box(title = 'Part Number is: '+ str(Select_A)+ '\n' + 'Work Hours',y = 'HrClocked',ylabel = 'Hours',xlabel='ManHour',legend = False, width = 500, height = 700)
    return ManHour
    
# For Manhour tab
p2 = pn.Row(
        pn.WidgetBox(Title,Select_A,Select_B),
        pn.Column(Hour_Plot),
        pn.bind(Day_Plot,Select_A,Select_B), name = 'ManHour Tracking')

# For Cost tab
p1 = pn.Row(
        pn.WidgetBox(Title,Select_X,Select_Y),
        pn.bind(Box_Plot,Select_X,Select_Y), name = 'Costs')


tabs = pn.Tabs(p1,p2, dynamic = True)
tabs


Tabs(dynamic=True)
    [0] Row(name='Costs')
        [0] WidgetBox
            [0] Markdown(str)
            [1] Select(name='Part Numbers', options=['505CH2', 2758, ...], value='505CH2')
            [2] Select(name='Work Required', options=['REP', 'O/H', 'SST'], value='REP')
        [1] ParamFunction(function)
    [1] Row(name='ManHour Tracking')
        [0] WidgetBox
            [0] Markdown(str)
            [1] Select(name='Part Numbers', options=['505CH2', 2758, ...], value='505CH2')
            [2] Select(name='Work Required', options=['REP', 'O/H', 'SST'], value='REP')
        [1] Column
            [0] ParamFunction(function)
        [2] ParamFunction(function)